# **ResNet**

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [7]:
# Hyper-parameters
num_epochs = 80
batch_size = 100
learning_rate = 0.001

**Device**

In [2]:
device = torch.device('gpu:0' if torch.cuda.is_available() else 'cpu')

**Image Preprocessing**

In [4]:
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()
])

**CIFAR-10 Dataset**

In [5]:
train_dataset = torchvision.datasets.CIFAR10(root='./data/',
                                             train=True, 
                                             transform=transform,
                                             download=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data/',
                                            train=False, 
                                            transform=transforms.ToTensor())

Files already downloaded and verified


**Data Loader**

In [8]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

**ResNet**

In [9]:
# 3x3 convolution
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3,
                     stride=stride, padding=1, bias=False)

In [11]:
# Residual block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

In [24]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], 2)
        self.layer3 = self.make_layer(block, 64, layers[2], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)
        
        
    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels)
            )
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [25]:
model = ResNet(ResidualBlock, [2, 2, 2]).to(device)

In [26]:
model

ResNet(
  (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): ResidualBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ResidualBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=

**Loss and Optimizer**

In [27]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

**Update Learning Rate**

In [28]:
# For updating learning rate
def update_lr(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

**Train Model**

In [29]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

    # Decay learning rate
    if (epoch+1) % 20 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)

Epoch [1/80], Step [100/500] Loss: 1.7530
Epoch [1/80], Step [200/500] Loss: 1.4055
Epoch [1/80], Step [300/500] Loss: 1.2324
Epoch [1/80], Step [400/500] Loss: 1.0779
Epoch [1/80], Step [500/500] Loss: 1.1011
Epoch [2/80], Step [100/500] Loss: 0.9274
Epoch [2/80], Step [200/500] Loss: 0.9632
Epoch [2/80], Step [300/500] Loss: 0.8786
Epoch [2/80], Step [400/500] Loss: 0.8255
Epoch [2/80], Step [500/500] Loss: 0.8333
Epoch [3/80], Step [100/500] Loss: 0.9297
Epoch [3/80], Step [200/500] Loss: 0.9137
Epoch [3/80], Step [300/500] Loss: 0.8262
Epoch [3/80], Step [400/500] Loss: 0.6748
Epoch [3/80], Step [500/500] Loss: 0.7276
Epoch [4/80], Step [100/500] Loss: 0.7367
Epoch [4/80], Step [200/500] Loss: 0.6881
Epoch [4/80], Step [300/500] Loss: 0.6155
Epoch [4/80], Step [400/500] Loss: 0.9518
Epoch [4/80], Step [500/500] Loss: 0.8719
Epoch [5/80], Step [100/500] Loss: 0.7550
Epoch [5/80], Step [200/500] Loss: 0.6498
Epoch [5/80], Step [300/500] Loss: 0.9098
Epoch [5/80], Step [400/500] Loss:

Epoch [39/80], Step [300/500] Loss: 0.2674
Epoch [39/80], Step [400/500] Loss: 0.1980
Epoch [39/80], Step [500/500] Loss: 0.2732
Epoch [40/80], Step [100/500] Loss: 0.1732
Epoch [40/80], Step [200/500] Loss: 0.2193
Epoch [40/80], Step [300/500] Loss: 0.2025
Epoch [40/80], Step [400/500] Loss: 0.2875
Epoch [40/80], Step [500/500] Loss: 0.2574
Epoch [41/80], Step [100/500] Loss: 0.1409
Epoch [41/80], Step [200/500] Loss: 0.1382
Epoch [41/80], Step [300/500] Loss: 0.1150
Epoch [41/80], Step [400/500] Loss: 0.0964
Epoch [41/80], Step [500/500] Loss: 0.2415
Epoch [42/80], Step [100/500] Loss: 0.2241
Epoch [42/80], Step [200/500] Loss: 0.2761
Epoch [42/80], Step [300/500] Loss: 0.2090
Epoch [42/80], Step [400/500] Loss: 0.1839
Epoch [42/80], Step [500/500] Loss: 0.3557
Epoch [43/80], Step [100/500] Loss: 0.1961
Epoch [43/80], Step [200/500] Loss: 0.2359
Epoch [43/80], Step [300/500] Loss: 0.1279
Epoch [43/80], Step [400/500] Loss: 0.1797
Epoch [43/80], Step [500/500] Loss: 0.2352
Epoch [44/8

Epoch [77/80], Step [400/500] Loss: 0.1058
Epoch [77/80], Step [500/500] Loss: 0.0688
Epoch [78/80], Step [100/500] Loss: 0.0758
Epoch [78/80], Step [200/500] Loss: 0.0886
Epoch [78/80], Step [300/500] Loss: 0.1515
Epoch [78/80], Step [400/500] Loss: 0.1994
Epoch [78/80], Step [500/500] Loss: 0.1632
Epoch [79/80], Step [100/500] Loss: 0.2257
Epoch [79/80], Step [200/500] Loss: 0.1221
Epoch [79/80], Step [300/500] Loss: 0.1588
Epoch [79/80], Step [400/500] Loss: 0.1492
Epoch [79/80], Step [500/500] Loss: 0.0917
Epoch [80/80], Step [100/500] Loss: 0.1427
Epoch [80/80], Step [200/500] Loss: 0.2102
Epoch [80/80], Step [300/500] Loss: 0.1249
Epoch [80/80], Step [400/500] Loss: 0.2286
Epoch [80/80], Step [500/500] Loss: 0.1637


**Test Model**

In [30]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

Accuracy of the model on the test images: 88.68 %


**Save Model**

In [31]:
# Save the model checkpoint
torch.save(model.state_dict(), 'resnet.ckpt')